# Logistic Regression in Spark

In [2]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'ddP5ODJp-sO3pM7HzxP8fbS0eACIZQBZwaRQMIrdCDkG',
    'service_id': 'iam-ServiceId-6afcc273-dc9c-4baf-97cc-aa2ce588f115',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_c926ba9e5f744993be30ee0b15b9ade4_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet(cos.url('hmp.parquet', 'advancedmlandsignalprocessing-donotdelete-pr-filqxc7nozcnt1'))
df.show()

+---+---+---+------------+--------------------+
|  x|  y|  z|       Class|              Source|
+---+---+---+------------+--------------------+
| 55| 13| 35|Climb_stairs|Accelerometer-201...|
| 12| 35| 36|Climb_stairs|Accelerometer-201...|
| 13| 34| 35|Climb_stairs|Accelerometer-201...|
| 12| 36| 36|Climb_stairs|Accelerometer-201...|
| 12| 36| 36|Climb_stairs|Accelerometer-201...|
| 12| 36| 35|Climb_stairs|Accelerometer-201...|
| 14| 36| 34|Climb_stairs|Accelerometer-201...|
| 12| 36| 35|Climb_stairs|Accelerometer-201...|
| 11| 36| 35|Climb_stairs|Accelerometer-201...|
| 12| 35| 36|Climb_stairs|Accelerometer-201...|
| 11| 34| 35|Climb_stairs|Accelerometer-201...|
| 12| 35| 37|Climb_stairs|Accelerometer-201...|
| 13| 35| 37|Climb_stairs|Accelerometer-201...|
| 10| 35| 36|Climb_stairs|Accelerometer-201...|
|  8| 35| 38|Climb_stairs|Accelerometer-201...|
|  9| 37| 38|Climb_stairs|Accelerometer-201...|
| 13| 35| 37|Climb_stairs|Accelerometer-201...|
| 13| 36| 37|Climb_stairs|Accelerometer-

In [8]:
# split the data frame into train and test sample
splits = df.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]
print(df_train.count())
print(df_test.count())

283385
70890


In [11]:
# build the stages of pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

indexer = StringIndexer(inputCol='Class',outputCol='label')
#encoder = OneHotEncoder(inputCol='')
vectorAssembler = VectorAssembler(inputCols = ['x','y','z'],outputCol = 'features')
normalizer = Normalizer(inputCol = 'features',outputCol='features_norm')
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [13]:
# build the pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])

In [14]:
# fit the pipeline to train data
model = pipeline.fit(df_train)

# predict using transform
prediction = model.transform(df_train)

In [15]:
# evaluate the performance of model (accuracy)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')

evaluator.evaluate(prediction)

0.12926583975863226

In [16]:
# evaluate on test dataframe
model_test = pipeline.fit(df_test)

In [17]:
prediction_test = model.transform(df_test)

In [18]:
evaluator.evaluate(prediction_test)

0.1293412328960361